In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time
import statsmodels.formula.api as sm
from statsmodels.formula.api import ols
import random

In [3]:
# Connect to an R session
import rpy2.robjects
r = rpy2.robjects.r
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula
from rpy2.robjects.environments import Environment
from rpy2.robjects import pandas2ri
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
#Specify path with downloaded R packages
lib_path = 'C:/Users/BUCKBEAK/Documents/BUCKBEAK/R/win-library/3.2'

# load some required packages
utils = importr('utils')
langR = importr('languageR', lib_loc=lib_path)
lme4 = importr('lme4', lib_path)
lmerTest=importr('lmerTest', lib_path)

#Bring in all datasets from regressions

In [6]:
#7 days weighted
df = pd.read_csv('final_race_df_7daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_7daysweighted.csv')
weight_regdf7 = pd.merge(df, df2, how='left')
weight_regdf7 = weight_regdf7.drop(df.columns[0], axis=1)

#6 days weighted
df = pd.read_csv('final_race_df_6daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_6daysweighted.csv')
weight_regdf6 = pd.merge(df, df2, how='left')
weight_regdf6 = weight_regdf6.drop(df.columns[0], axis=1)

#5 days weighted
df = pd.read_csv('final_race_df_5daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_5daysweighted.csv')
weight_regdf5 = pd.merge(df, df2, how='left')
weight_regdf5 = weight_regdf5.drop(df.columns[0], axis=1)

#4 days weighted
df = pd.read_csv('final_race_df_4daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_4daysweighted.csv')
weight_regdf4 = pd.merge(df, df2, how='left')
weight_regdf4 = weight_regdf4.drop(df.columns[0], axis=1)

#3 days weighted
df = pd.read_csv('final_race_df_6daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_6daysweighted.csv')
weight_regdf3 = pd.merge(df, df2, how='left')
weight_regdf3 = weight_regdf3.drop(df.columns[0], axis=1)

#2 days weighted
df = pd.read_csv('final_race_df_6daysweighted.csv')
df2 = pd.read_csv('workout_final_race_df_6daysweighted.csv')
weight_regdf2 = pd.merge(df, df2, how='left')
weight_regdf2 = weight_regdf2.drop(df.columns[0], axis=1)

#DataFrames for the 5 individual days before the race
df = pd.read_csv('race_df_pre_weighting.csv')
dummies = pd.get_dummies(df['user_id'], prefix='user')
temp_df = pd.concat([df, dummies], axis=1)
race_times = pd.read_csv('huxc_race_times.csv')
race_times=race_times[race_times['user_id']!=2461]
race_times=race_times[race_times['user_id']!=2509]
reg_df = temp_df.merge(race_times, how='left')
reg_df = reg_df[reg_df['user_id']!=2439]
reg_df = reg_df.drop(['user_828'], axis=1)
reg_df = reg_df.drop(df.columns[0], axis=1).reset_index(drop=True)

reg_df0 = reg_df[reg_df['buildup_days']==0].reset_index(drop=True)
reg_df1 = reg_df[reg_df['buildup_days']==1].reset_index(drop=True)
reg_df2 = reg_df[reg_df['buildup_days']==2].reset_index(drop=True)
reg_df3 = reg_df[reg_df['buildup_days']==3].reset_index(drop=True)
reg_df4 = reg_df[reg_df['buildup_days']==4].reset_index(drop=True)
reg_df5 = reg_df[reg_df['buildup_days']==5].reset_index(drop=True)

#Exponentially Weighted DataFrames
rec_weight_df5 = pd.read_csv('expo_final_race_df_5daysweighted.csv')
wkout_weight_df5 = pd.read_csv('expo_workout_final_race_df_5daysweighted.csv')
expo_weight_df5 = pd.merge(rec_weight_df5, wkout_weight_df5, how='left')
expo_weight_df5 = expo_weight_df5.drop(df.columns[0], axis=1)

rec_weight_df4 = pd.read_csv('expo_final_race_df_4daysweighted.csv')
wkout_weight_df4 = pd.read_csv('expo_workout_final_race_df_4daysweighted.csv')
expo_weight_df4 = pd.merge(rec_weight_df4, wkout_weight_df4, how='left')
expo_weight_df4 = expo_weight_df4.drop(df.columns[0], axis=1)

rec_weight_df3 = pd.read_csv('expo_final_race_df_3daysweighted.csv')
wkout_weight_df3 = pd.read_csv('expo_workout_final_race_df_3daysweighted.csv')
expo_weight_df3 = pd.merge(rec_weight_df3, wkout_weight_df3, how='left')
expo_weight_df3 = expo_weight_df3.drop(df.columns[0], axis=1)

rec_weight_df2 = pd.read_csv('expo_final_race_df_2daysweighted.csv')
wkout_weight_df2 = pd.read_csv('expo_workout_final_race_df_2daysweighted.csv')
expo_weight_df2 = pd.merge(rec_weight_df2, wkout_weight_df2, how='left')
expo_weight_df2 = expo_weight_df2.drop(df.columns[0], axis=1)

Then grabbing variables that were significant, and put into one dataframe

- REM sleep, runner mixed effects, two nights before the race: significant for both Bonferroni and Permutation Correction
- REM sleep, runner mixed effects, weighted two days: significant for Bonferroni only
- REM sleep, runner mixed effects, weighted three days: significant for Bonferroni only
- Total sleep, runner mixed effects, weighted three days: significant for Bonerroni only
- REM sleep, runner mixed effects, weighted four days: significant for Bonerroni only
- 

#The Functions We Need From Previous Notebooks

In [4]:
def mixed_effects_model(dftouse, env, y_var, x_var, group_vars=None):
    env = env
    if group_vars == None:
        print 'Not a mixed effects model!'
        return 'Not a mixed effects model!'
    for varname in r.colnames(dftouse):
        env[varname] = dftouse.rx2(varname)
    if type(group_vars) == str:
        formula = Formula(y_var + ' ~ ' + x_var + ' + (1|' + group_vars + ')', environment = env)
        model = lmerTest.lmer(formula)
        return r.summary(model)
    elif type(group_vars)== tuple or type(group_vars)== list:
        if len(group_vars) == 2:
            formula = Formula(y_var + ' ~ ' + x_var + ' + (1|' + group_vars[0] + ') + (1|' + group_vars[1] + ')', environment=env)
            model = lmerTest.lmer(formula)
            return r.summary(model)
        elif len(group_vars) == 3:
            formula =Formula(y_var + ' ~ ' + x_var + ' + (1|' + group_vars[0] + ' ' + group_vars[1] + ' ' + group_vars[2]+ ')', 
                             environment = env)
            model = lmerTest.lmer(formula)
            return r.summary(model)
    

In [5]:
#Just playing with the 5 weighted day variable
pandas2ri.activate()
rec_weight_df5 = pd.read_csv('final_race_df_5daysweighted.csv')
wkout_weight_df5 = pd.read_csv('workout_final_race_df_5daysweighted.csv')
weight_df5 = pd.merge(rec_weight_df5, wkout_weight_df5, how='left')
r_weightdf5 = pandas2ri.py2ri(weight_df5)
reg_test=r_weightdf5

#Trying a Stepwise Model with all variables

However, this was leading to errors

In [43]:
env=Environment()
xlist = ['hrv_rmssd', 'resting_heart_rate', 'light_sleep_duration', 'rem_sleep_duration', 'slow_wave_sleep_duration',
'cycles_count', 'latency', 'z1', 'z2', 'z3', 'z4', 'z5']
for varname in r.colnames(reg_test):
    env[varname] = reg_test.rx2(varname)
x_string= ' + '
for i in range(0, len(xlist)):
    if i != (len(xlist)-1):
        x_string+= xlist[i] + ' + '
    else:
        x_string+= xlist[i] + ' '
formula = Formula('pace_per_k' + ' ~ ' + x_string + ' + (1|user_id)', environment = env)
fit = lmerTest.lmer(formula)
stepwise_model = r.step(fit, direction='both')

RRuntimeError: Error in solve.default(xtx[inds, inds]) : 
  system is computationally singular: reciprocal condition number = 5.51812e-19


In [20]:
xlist = ['hrv_rmssd', 'resting_heart_rate', 'light_sleep_duration', 'rem_sleep_duration', 'sleep_duration', 'slow_wave_sleep_duration',
'cycles_count', 'time_in_bed', 'latency', 'z1', 'z2', 'z3', 'z4', 'z5']

In [25]:
def stepwise_model(dftouse, env, y_var, x_vars, group_vars=None):
    if group_vars==None:
        print 'Not a mixed effects model!'
        return 'Not a mixed effects model!'
    env=env
    for varname in r.colnames(dftouse):
        env[varname] = dftouse.rx2(varname)
    
    x_string= ' + '
    for i in range(0, len(xlist)):
        if i != (len(xlist)-1):
            x_string+= xlist[i] + ' + '
        else:
            x_string+= xlist[i] + ' '
    
    if type(group_vars) == str:
        formula = Formula(y_var + ' ~ ' + x_string + ' + (1|' + group_vars + ')', environment = env)
        fit = lmerTest.lmer(formula)
        return fit
    
    elif type(group_vars)== tuple or type(group_vars)== list:
        if len(group_vars) == 2:
            formula = Formula(y_var + ' ~ ' + x_string + ' + (1|' + group_vars[0] + ') + (1|' + group_vars[1] + ')', environment=env)
            fit = lmerTest.lmer(formula)
            return fit
        
        elif len(group_vars) == 3:
            formula =Formula(y_var + ' ~ ' + x_string + ' + (1|' + group_vars[0] + ' ' + group_vars[1] + ' ' + group_vars[2]+ ')', 
                             environment = env)
            fit = lmerTest.lmer(formula)
            return fit

In [26]:
env=Environment()
fit = stepwise_model(reg_test, env, 'pace_per_k', xlist, group_vars='user_id')

RRuntimeError: Error: Dropping columns failed to produce full column rank design matrix
